In [1]:
import pandas as pd
import numpy as np

import yfinance as yf
import pandas_datareader as pdr

# Inflation Data

In [2]:
STARTDATE = '2015-01-01'
ENDDATE = '2025-05-31'

startdate = pd.to_datetime(STARTDATE)
enddate = pd.to_datetime(ENDDATE)

ticks = ['CPIAUCSL']
labels = ['inflation']

rawdata = pdr.DataReader(ticks,'fred',startdate,enddate)
rawdata.index.name = 'date'
rawdata.index = pd.to_datetime(rawdata.index)
rawdata.columns = labels

In [3]:
FREQ = 12

if FREQ == 4:
    FREQcode = 'Q'
elif FREQ == 1:
    FREQcode = 'Y'
elif FREQ==12:
    FREQcode = 'M'

cpi = rawdata.resample(FREQcode).agg('last')
cpi.dropna(inplace=True)


HORZ = FREQ
inflation = (cpi / cpi.shift(HORZ) -1).dropna()

inflation.index.name = 'date'

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_24392/377805030.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  cpi = rawdata.resample(FREQcode).agg('last')


# Security Returns

In [4]:
tickers = ['SPY','EFA','EEM','PSP','QAI','HYG','DBC','IYR','IEF','BWX','TIP']

df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']

prices = df.resample('M').last()
rets = prices.pct_change()

# keep simple by dropping months where any security has nan
rets.dropna(inplace=True)
# reorder columns to match info tab, not alphabetical
rets = rets[tickers]

rets.index.name = 'date'

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  11 of 11 completed
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_24392/3665669303.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


# Align

In [5]:
inflation, rets = inflation.align(rets,join='inner',axis=0)

# Export Data

In [6]:
with pd.ExcelWriter('../data/exam_3_data.xlsx') as writer:  
#    inflation.to_excel(writer, sheet_name='inflation')
    rets.to_excel(writer, sheet_name='returns')